be really clear about what's going into repaired tables: 
- repaired
- already OK
- unrepairable

Only ~ 215k repaired `env_package`s ?!
- yeah, there just aren't that many annotated in the biosample metadata


- some of this code should probably be moved back into the package
    - splitting and repiping 

- read in previous SSSOMs and don't re-map? or at least double check?

- don't forget to consult "model" field in biosample table too
    - for scoping, in addition to en_package and taxonomy_id?

- what to do about hits against obsolete classes... does OLS provide a mechanism to avoid?
- be careful with GAZ matches
- avoid IDless, OMIT, NCIT, MICRO, BTO

- is it worth trimming and concatenating "x1", "x2", "x3", where x is a mappable term? numbers may be low.
    - example = XXX

- reconcile "x biome" and "x" mappings
- take note of non-local salvage hits
    - like XXX
    - small/large (freshwater) lake?
    - include native labels in the review frame, in addition to local?
    - should use rdftab, not additional OLS calls
    

- switch from hard-coded `env_package_overrides` to something like SSSOM?
- mappings can come from merging strategies, first pass OLS or salvage OLS
- reflect those methods in SSSOM
- add string distance/confidence
    - multiple columns for multiple methods?

review these SSSOM columns

```
    "subject_category": [biosample_col_to_map] * results_rows,
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
```

- are there samples with a host_taxid and a (sample) taxonomy_id from "unclassified"?
- start at uncultured or metagenome taxonomy_id?

- what scoping on or mapping of  "gold" or "path"? WON'T FIND THAT IN NCBI BIOSAMPLE


What to save where? 
- new columns in biosample sqlite? 
- delimited file dumps
- move content from original column and insert the new values into that original column?
- pandas clipboard integration handy for review step


In [1]:
import pandas as pd
import re
import requests as requests
import sqlite3
import string
import urllib
import yaml

import scoped_mapping

from datetime import datetime
from pkg_resources import get_distribution, DistributionNotFound
from strsimpy.cosine import Cosine
from xml.etree import ElementTree
from tdda import rexpy

# User-provided data
See repo README for notes on setting up SQLite databases of OBO ontologies with semantic-sql, relation-graph and rdftab

In [2]:
# from https://www.ncbi.nlm.nih.gov/biosample/docs/packages/?format=xml
# see also https://www.ncbi.nlm.nih.gov/biosample/docs/packages/
biosample_packages_file = "../target/biosample_packages.xml"

# from ftp://ftp.ncbi.nlm.nih.gov//biosample/biosample_set.xml.gz
# via harmonized_table.db.gz
# in https://drive.google.com/drive/u/0/folders/1eL0v0stoduahjDpoDJIk3z2pJBAU4b2Y
biosample_sqlite_file = "../target/harmonized_table.db"

# where do we require a single ontology and where can we use multiple?
target_onto_prefix = "PO"

first_pass_ontologies = ["envo", "ncbitaxon"]

----

## It's nice to see everything accounted for
(In terms of manual mappings for env_package.) But we can prioritize NMDC for now:

- Soil 15,777
- Sediment 7,147
- Plant-associated 3,142

Could some of these "no environmental package" mappings be losing important granularity?

Map `None` and '' to "no environmental package"?

In [3]:
env_package_overrides = {
    "built environment": "built",
    "misc environment": "miscellaneous",
    "missing": "no environmental package",
    "unknown": "no environmental package",
    "default": "no environmental package",
    "unspecified": "no environmental package",
    "not available": "no environmental package",
    "not collected": "no environmental package",
    "miscellaneous natural or artificial environment": "miscellaneous",
    "not applicable": "no environmental package",
    "soil-associated": "soil",
    "soil associated": "soil",
}

In [4]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)

first_pass_ontologies.insert(0, target_onto_prefix.lower())
first_pass_ontologies_str = ",".join(first_pass_ontologies)

## Determine ID patterns for common ontologies, like `ENVO`

In [5]:
# first_pass_ontologies, including primary but not ncbitaxon
# doesn't check for exisitence of DB files

temp = first_pass_ontologies
temp = [each_string.lower() for each_string in temp]
temp.sort()
temp = [each_string for each_string in temp if each_string != "ncbitaxon"]

print(temp)

first_pass_id_frames = []
for one_ontology in temp:
    print(one_ontology)
    one_db_file = "../semantic-sql/db/" + one_ontology.lower() + ".db"
    #     print(one_db_file)
    one_con = sqlite3.connect(one_db_file)

    # FIXED? this wont include a term unless it's a class with a label
    # add obsolete tags?
    # may want to make a local-label only frame for later tasks?

    q = """
select
	distinct s1.stanza,
	s2.value
from
	statements s1
left join statements s2 on
	s2.subject = s1.subject
where
	s1.predicate = 'rdf:type'
	and s1.object = 'owl:Class'
	and s1.stanza = s1.subject
	and s2.predicate = 'rdfs:label'"""
    [ids_labs_selected_ontolgies, query_duration] = scoped_mapping.timed_query(
        q, one_con
    )
    print(query_duration)
    ids_labs_selected_ontolgies["ontology"] = one_ontology
    first_pass_id_frames.append(ids_labs_selected_ontolgies)
    one_con.close()

['envo', 'po']
envo
0:00:00.460708
po
0:00:00.054062


In [6]:
ids_labs_selected_ontolgies = pd.concat(first_pass_id_frames)
ids_labs_selected_ontolgies.to_sql(
    "ids_labs_selected_ontolgies", biosample_cnx, if_exists="replace", index=False
)

ids_labs_selected_ontolgies

,stanza,value,ontology
0,BFO:0000001,entity,envo
1,BFO:0000002,continuant,envo
2,BFO:0000003,occurrent,envo
3,BFO:0000004,independent continuant,envo
4,BFO:0000006,spatial region,envo
...,...,...,...
1783,PO:0030132,cincinnus inflorescence,po
1784,PO:0030133,compound cincinnus inflorescence,po
1785,PO:0030134,bostryx inflorescence,po
1786,PO:0030135,drepanium inflorescence,po


In [7]:
ids_labs_selected_ontolgies = scoped_mapping.add_prefix_col(
    ids_labs_selected_ontolgies, "stanza", "prefix"
)
ids_labs_selected_ontolgies

,stanza,value,ontology,prefix
0,BFO:0000001,entity,envo,BFO
1,BFO:0000002,continuant,envo,BFO
2,BFO:0000003,occurrent,envo,BFO
3,BFO:0000004,independent continuant,envo,BFO
4,BFO:0000006,spatial region,envo,BFO
...,...,...,...,...
1783,PO:0030132,cincinnus inflorescence,po,PO
1784,PO:0030133,compound cincinnus inflorescence,po,PO
1785,PO:0030134,bostryx inflorescence,po,PO
1786,PO:0030135,drepanium inflorescence,po,PO


In [8]:
id_patterns = scoped_mapping.get_multi_term_patterns(
    ids_labs_selected_ontolgies, "stanza", "prefix"
)

id_patterns

{'BFO': 'BFO:\\d{7}',
 'CARO': 'CARO:\\d{7}',
 'CHEBI': 'CHEBI:\\d{4,6}',
 'ENVO': 'ENVO:\\d{7,8}',
 'FAO': 'FAO:0000001',
 'FOODON': 'FOODON:\\d{8}',
 'GO': 'GO:\\d{7}',
 'IAO': 'IAO:\\d{7}',
 'NCBITaxon': 'NCBITaxon:\\d+',
 'OBI': 'OBI:\\d{7}',
 'PATO': 'PATO:\\d{7}',
 'PCO': 'PCO:\\d{7}',
 'PO': 'PO:\\d{7}',
 'RO': 'RO:0002577',
 'UBERON': 'UBERON:\\d{7}'}

In [9]:
ipf = pd.DataFrame(id_patterns.items(), columns=["ontology", "id_pattern"])
ipf.to_sql("id_patterns", biosample_cnx, if_exists="replace", index=False)

In [10]:
# # round trip
# ipd = dict(zip(ipf.ontology, ipf.id_pattern))

## Retreive `env_package` values from Biosample table

In [11]:
q = """
select
    env_package,
    count(*) as count
from
    biosample b
group by
    env_package
order by
    count(*) desc
"""
[env_package_count, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

env_package_count

0:00:01.100606


,env_package,count
0,None,14083847
1,host-associated,49254
2,human-gut,47921
3,water,16367
4,human-skin,13706
...,...,...
87,env_package,1
88,gut microbiome,1
89,marine sediment (ENVO:00002113),1
90,saliva,1


## Apply some normalization rules to the `env_package` values

In [12]:
env_package_normalization = scoped_mapping.env_package_nomralizastion(
    env_package_count, "env_package", id_patterns[target_onto_prefix]
)

# getting rid of redundant? 'string' column
env_package_normalization = env_package_normalization[
    [
        "env_package",
        "count",
        "lhs",
        "rhs",
        "extract",
        "remaining_string",
        "remaining_tidied",
    ]
]

env_package_normalization

,env_package,count,lhs,rhs,extract,remaining_string,remaining_tidied
0,None,14083847,,None,,,
1,host-associated,49254,,host-associated,,host-associated,host associated
2,human-gut,47921,,human-gut,,human-gut,human gut
3,water,16367,,water,,water,water
4,human-skin,13706,,human-skin,,human-skin,human skin
...,...,...,...,...,...,...,...
87,env_package,1,,env_package,,env_package,env package
88,gut microbiome,1,,gut microbiome,,gut microbiome,gut microbiome
89,marine sediment (ENVO:00002113),1,,marine sediment (ENVO:00002113),,marine sediment (ENVO:00002113),marine sediment envo 00002113
90,saliva,1,,saliva,,saliva,saliva


In [13]:
package_dictionary = scoped_mapping.get_package_dictionary(biosample_packages_file)
package_dictionary.to_sql(
    "package_dictionary", biosample_cnx, if_exists="replace", index=False
)
package_dictionary

,Name,DisplayName,ShortName,EnvPackage,EnvPackageDisplay,NotAppropriateFor,Description,Example
0,Generic.1.0,Generic,,,,,Generic,
1,SARS-CoV-2.cl.1.0,SARS-CoV-2: clinical or host-associated; versi...,SARS-CoV-2: clinical or host-associated,,,wgs_single;wgs_batch;wgs_diploid,Use for SARS-CoV-2 samples that are relevant t...,
2,Pathogen.cl.1.0,Pathogen: clinical or host-associated; version...,Pathogen: clinical or host-associated,,,,Clinical or host-associated pathogen,SAMN02928182
3,Pathogen.env.1.0,Pathogen: environmental/food/other; version 1.0,Pathogen: environmental/food/other,,,,"Environmental, food or other pathogen",SAMN02730065
4,Microbe.1.0,Microbe; version 1.0,Microbe,,,,Use for bacteria or other unicellular microbes...,SAMN02911891
...,...,...,...,...,...,...,...,...
149,MIUVIG.plant-associated.5.0,"MIUVIG: uncultivated virus genome, plant-assoc...",MIUVIG Uncultivated Virus Genome,plant-associated,plant-associated,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
150,MIUVIG.sediment.5.0,"MIUVIG: uncultivated virus genome, sediment; v...",MIUVIG Uncultivated Virus Genome,sediment,sediment,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
151,MIUVIG.soil.5.0,"MIUVIG: uncultivated virus genome, soil; versi...",MIUVIG Uncultivated Virus Genome,soil,soil,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,
152,MIUVIG.wastewater.5.0,"MIUVIG: uncultivated virus genome, wastewater;...",MIUVIG Uncultivated Virus Genome,wastewater,wastewater,wgs_single;wgs_batch;wgs_diploid,Use for uncultivated virus genome identified i...,


pick "EnvPackage"/"eptidy" from package dictionary as canonical, not "EnvPackageDisplay"/"epdtidy"


But still want to support making XXX values from YYY canonical according to "EnvPackageDisplay"/"epdtidy", so make a mapping/override table

In [14]:
package_dictionary = scoped_mapping.make_tidy_col(
    package_dictionary, "EnvPackage", "eptidy"
)
package_dictionary = scoped_mapping.make_tidy_col(
    package_dictionary, "EnvPackageDisplay", "epdtidy"
)

# update in sqlite
package_dictionary.to_sql(
    "package_dictionary", biosample_cnx, if_exists="replace", index=False
)

In [15]:
epd_to_ep = package_dictionary[["eptidy", "epdtidy"]]
# drop duplicates
epd_to_ep = epd_to_ep.drop_duplicates()

# drop blank eptidy rows
ep_blank_flag = epd_to_ep["eptidy"].eq("")
epd_to_ep = epd_to_ep.loc[~ep_blank_flag]

# drop rows where eptidy and epdtidy are the same
identical_flag = epd_to_ep["eptidy"] == epd_to_ep["epdtidy"]
epd_to_ep = epd_to_ep.loc[~identical_flag]

epd_to_ep

,eptidy,epdtidy
21,microbial,microbial mat biofilm
22,miscellaneous,miscellaneous or artificial
26,wastewater,wastewater sludge


In [16]:
# and add to manualy asserted overrides above
overrides_supplement = dict(zip(epd_to_ep["epdtidy"], epd_to_ep["eptidy"]))

overrides_supplement

{'microbial mat biofilm': 'microbial',
 'miscellaneous or artificial': 'miscellaneous',
 'wastewater sludge': 'wastewater'}

In [17]:
env_package_overrides.update(overrides_supplement)

env_package_overrides

{'built environment': 'built',
 'misc environment': 'miscellaneous',
 'missing': 'no environmental package',
 'unknown': 'no environmental package',
 'default': 'no environmental package',
 'unspecified': 'no environmental package',
 'not available': 'no environmental package',
 'not collected': 'no environmental package',
 'miscellaneous natural or artificial environment': 'miscellaneous',
 'not applicable': 'no environmental package',
 'soil-associated': 'soil',
 'soil associated': 'soil',
 'microbial mat biofilm': 'microbial',
 'miscellaneous or artificial': 'miscellaneous',
 'wastewater sludge': 'wastewater'}

In [18]:
env_package_normalization = scoped_mapping.add_overrides(
    env_package_normalization, "remaining_tidied", "rt_override", env_package_overrides
)

In [19]:
denorm_frame = package_dictionary[["EnvPackage", "eptidy"]]
denorm_frame = denorm_frame.drop_duplicates()
denorm_frame

,EnvPackage,eptidy
0,,
12,No environmental package,no environmental package
13,air,air
14,built,built
15,host-associated,host associated
16,human-associated,human associated
17,human-gut,human gut
18,human-oral,human oral
19,human-skin,human skin
20,human-vaginal,human vaginal


In [20]:
env_package_normalization = env_package_normalization.merge(
    denorm_frame, how="left", left_on="rt_override", right_on="eptidy"
)

env_package_normalization = env_package_normalization[
    [
        "env_package",
        "count",
        "lhs",
        "rhs",
        "extract",
        "remaining_string",
        "remaining_tidied",
        "rt_override",
        "EnvPackage",
    ]
]

non_canonical_flag = env_package_normalization["EnvPackage"].isna()
env_package_normalization["is_canonical"] = True
env_package_normalization.loc[non_canonical_flag, "is_canonical"] = False

- env_package = env_package annotation from NCBI Biosample file XXX
- count = number of biosamples using that env_package annotation
- lhs = checklist info
- rhs = potential package info
- extract = potential OBO ID from rhs column (currently harcoded and only looking for ENVO IDs)
- remaining_string = rhs/string, with potential OBO IDs removed
- remaining_tidied = remaining_string with case, whitespace and punctuation normailzastion
- rt_override = some remaining_tidied values can be replaced according to env_package_overrides
- EnvPackage = corresponding de-normalized value from package_dictionary
- is_canonical = false when EnvPackage is NaN

In [21]:
env_package_normalization.to_sql(
    "env_package_normalization", biosample_cnx, if_exists="replace", index=False
)

In [22]:
env_package_normalization

,env_package,count,lhs,rhs,extract,remaining_string,remaining_tidied,rt_override,EnvPackage,is_canonical
0,None,14083847,,None,,,,,,True
1,host-associated,49254,,host-associated,,host-associated,host associated,host associated,host-associated,True
2,human-gut,47921,,human-gut,,human-gut,human gut,human gut,human-gut,True
3,water,16367,,water,,water,water,water,water,True
4,human-skin,13706,,human-skin,,human-skin,human skin,human skin,human-skin,True
...,...,...,...,...,...,...,...,...,...,...
87,env_package,1,,env_package,,env_package,env package,env package,NaN,False
88,gut microbiome,1,,gut microbiome,,gut microbiome,gut microbiome,gut microbiome,NaN,False
89,marine sediment (ENVO:00002113),1,,marine sediment (ENVO:00002113),,marine sediment (ENVO:00002113),marine sediment envo 00002113,marine sediment envo 00002113,NaN,False
90,saliva,1,,saliva,,saliva,saliva,saliva,NaN,False


## What do the successful normalizations look like?

In [23]:
q = """
select
    env_package,
    count,
    lhs,
    extract,
    EnvPackage
from
    env_package_normalization
where
    is_canonical = 1
"""
[successful_normalizastions, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx
)

print(query_duration)

successful_normalizastions

0:00:00.001661


,env_package,count,lhs,extract,EnvPackage
0,None,14083847,,,
1,host-associated,49254,,,host-associated
2,human-gut,47921,,,human-gut
3,water,16367,,,water
4,human-skin,13706,,,human-skin
...,...,...,...,...,...
56,Water,3,,,water
57,not collected,3,,,No environmental package
58,host associated,2,,,host-associated
59,Sediment,1,,,sediment


# Are there any normalization failures?

In [24]:
q = """
select
    env_package,
    count,
    lhs,
    extract,
    EnvPackage
from
    env_package_normalization
where
    is_canonical = 0
"""
[normalizastion_failures, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

normalizastion_failures

0:00:00.001545


,env_package,count,lhs,extract,EnvPackage
0,mimarks,479,,,None
1,mouse-gut,406,,,None
2,gut,172,,,None
3,biofilm,114,,,None
4,human-not providedsopharyngeal,107,,,None
5,mice gut,87,,,None
6,CV,60,,,None
7,"home, outdoor environment",44,,,None
8,fermentation-associated,42,,,None
9,sterile water,35,,,None


In [25]:
cursor = biosample_cnx.cursor()
statement = """
DROP TABLE if exists repaired_env_package ;

CREATE TABLE repaired_env_package AS
select
	b.id,
	epn.env_package as env_package_orig,
	epn.EnvPackage as env_package_rep
from
	env_package_normalization epn
join biosample b on
	b.env_package = epn.env_package
where
	is_canonical = 1
	and EnvPackage != '';
	
select
	count(*)
from
	repaired_env_package rep
"""

cursor.executescript(statement)
biosample_cnx.commit()